# Build Percentiles Analysis

This notebook mirrors the functionality in `build_percentiles.py` to compute delay percentiles by route, hour, and day of week.

**Note:** Make sure to install dependencies first:
```bash
pip install -r ../requirements.txt
```


In [ ]:
# Imports
import pandas as pd
import os
import glob
from pathlib import Path

# Set working directory to project root
os.chdir('..')
print(f"Working directory: {os.getcwd()}")


In [ ]:
# Load and concatenate all daily delay files
data_files = glob.glob("data/raw_delays/delays_*.csv.gz")
print(f"Found {len(data_files)} data files:")
for file in data_files:
    print(f"  - {file}")

if not data_files:
    print("No data files found. Run pipeline.py first!")
else:
    # Load all files
    all_data = []
    for file in data_files:
        df = pd.read_csv(file, compression="gzip")
        all_data.append(df)
        print(f"Loaded {file}: {len(df)} rows")
    
    # Combine all data
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"\nCombined dataset: {len(combined_df)} total rows")
    
    # Show sample
    print("\nSample data:")
    display(combined_df.head())


In [ ]:
# Compute percentiles (same logic as build_percentiles.py)
if 'combined_df' in locals() and not combined_df.empty:
    # Derive hour and day of week
    combined_df["hour"] = combined_df["gbtt_pta"].str[:2].astype(int)
    combined_df["date_dt"] = pd.to_datetime(combined_df["date"])
    combined_df["dow"] = combined_df["date_dt"].dt.dayofweek
    
    # Filter valid data
    filtered_df = combined_df.dropna(subset=["origin", "dest", "hour", "dow", "delay_min"])
    filtered_df = filtered_df[filtered_df["delay_min"] >= 0]
    
    print(f"Filtered dataset: {len(filtered_df)} valid rows")
    
    # Group by route, hour, and day of week
    grouped = filtered_df.groupby(["origin", "dest", "hour", "dow"])["delay_min"]
    
    # Calculate percentiles
    percentiles_df = grouped.agg([
        ("p80", lambda x: x.quantile(0.80)),
        ("p90", lambda x: x.quantile(0.90)),
        ("p95", lambda x: x.quantile(0.95)),
        ("obs_count", "count")
    ]).reset_index()
    
    # Flatten column names
    percentiles_df.columns = ["origin", "dest", "hour", "dow", "p80", "p90", "p95", "obs_count"]
    
    # Sort and round
    percentiles_df = percentiles_df.sort_values(["origin", "dest", "hour", "dow"]).reset_index(drop=True)
    for col in ["p80", "p90", "p95"]:
        percentiles_df[col] = percentiles_df[col].round(2)
    
    print(f"\nGenerated {len(percentiles_df)} percentile groups")
    print(f"Unique routes: {percentiles_df[['origin', 'dest']].drop_duplicates().shape[0]}")
    print(f"Total observations: {percentiles_df['obs_count'].sum()}")
else:
    print("No data to process")


In [ ]:
# Write CSV and display sample
if 'percentiles_df' in locals() and not percentiles_df.empty:
    # Save to CSV
    output_path = "data/route_hour_p80_p90_p95.csv"
    os.makedirs("data", exist_ok=True)
    percentiles_df.to_csv(output_path, index=False)
    
    print(f"✅ Saved percentiles to {output_path}")
    
    # Display sample of results
    print("\nSample percentile data:")
    display(percentiles_df.head(10))
    
    # Summary statistics
    print("\nSummary by route:")
    route_summary = percentiles_df.groupby(['origin', 'dest']).agg({
        'obs_count': 'sum',
        'p90': 'mean'
    }).round(2).reset_index()
    route_summary.columns = ['Origin', 'Dest', 'Total Obs', 'Avg P90']
    display(route_summary)
else:
    print("No percentiles data to save")
